In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import classification_report


from sklearn.utils import resample

In [2]:
df = pd.read_csv('../data/waterQuality1.csv')
# https://www.kaggle.com/datasets/mssmartypants/water-quality/data

In [3]:
df = df.drop(df[df['ammonia'] == "#NUM!"].index)
df['ammonia'] = df['ammonia'].astype('float')
df['is_safe'] = df['is_safe'].astype('float')

In [4]:
# df.info()

In [5]:
df_majority = df[df.is_safe == 0]
df_minority = df[df.is_safe == 1]

 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=1000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.is_safe.value_counts()
df_downsampled

,aluminium,ammonia,arsenic,barium,cadmium,chloramine,chromium,copper,flouride,bacteria,...,lead,nitrates,nitrites,mercury,perchlorate,radium,selenium,silver,uranium,is_safe
6721,0.08,22.70,0.05,0.61,0.090,0.03,0.04,0.61,1.46,0.84,...,0.136,16.56,1.62,0.007,7.88,0.10,0.06,0.07,0.08,0.0
7928,0.03,8.49,0.05,1.89,0.090,0.19,0.05,0.09,1.42,0.67,...,0.194,1.50,1.49,0.004,0.90,2.94,0.01,0.05,0.09,0.0
2128,0.02,17.84,0.01,3.94,0.008,2.26,0.03,1.40,0.50,0.28,...,0.044,2.71,1.80,0.001,1.02,7.90,0.04,0.28,0.03,0.0
5970,0.02,0.72,0.00,1.58,0.060,0.44,0.00,1.40,0.44,0.07,...,0.150,10.90,0.96,0.005,2.68,3.43,0.02,0.03,0.08,0.0
5181,0.08,1.73,0.07,1.89,0.070,0.09,0.06,1.33,1.00,0.00,...,0.152,17.00,0.08,0.006,2.94,1.23,0.07,0.09,0.06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7994,0.05,7.78,0.00,1.95,0.040,0.10,0.03,0.03,1.37,0.00,...,0.197,14.29,1.00,0.005,3.57,2.13,0.09,0.06,0.03,1.0
7995,0.05,24.22,0.02,0.59,0.010,0.45,0.02,0.02,1.48,0.00,...,0.031,10.27,1.00,0.001,1.48,1.11,0.09,0.10,0.08,1.0
7996,0.09,6.85,0.00,0.61,0.030,0.05,0.05,0.02,0.91,0.00,...,0.182,15.92,1.00,0.000,1.35,4.84,0.00,0.04,0.05,1.0
7997,0.01,10.00,0.01,2.00,0.000,2.00,0.00,0.09,0.00,0.00,...,0.000,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,1.0


In [6]:

x = np.array(df_downsampled.drop(['is_safe'], axis=1))
y = np.array(df_downsampled['is_safe'].copy())

from model_selection import positive_negative_check
positive_negative_check(y)

y = np.where(y <= 0, -1, 1)
positive_negative_check(y)

Positive Class [1.0]: 912 sample.
Negative Class [0.0]: 1000 sample.
Total 1912 Samples.
Positive Class [1]: 912 sample.
Negative Class [-1]: 1000 sample.
Total 1912 Samples.


In [7]:
# from model_selection import train_test_split
from sklearn.model_selection import train_test_split
# buff_x, x_test, buff_y, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
# x_train, x_validate, y_train, y_validate = train_test_split(buff_x,buff_y, test_size=0.2, random_state=42)

buff_x, x_test, buff_y, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
x_train, x_validate, y_train, y_validate = train_test_split(buff_x,buff_y, test_size=0.3, random_state=11)
# x_train

In [8]:
model = svm.SVC(kernel='linear' ,C=10)
model.fit(x_train, y_train)

model.score(x_validate, y_validate)
y_pred = model.predict(x_test)
print(y_pred)
model.score(x_test, y_test)
    


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print("Precision:",metrics.precision_score(y_test, y_pred, average="weighted"))
# print("Recall:",metrics.f1_score(y_test, y_pred, average="macro"))
# print(classification_report(y_test, y_pred))


[ 1 -1 -1  1 -1  1 -1 -1 -1  1  1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1  1  1
 -1  1 -1  1  1  1 -1 -1  1 -1  1  1 -1 -1 -1  1 -1  1 -1  1  1 -1 -1 -1
 -1  1  1 -1 -1 -1 -1 -1  1 -1 -1  1  1  1  1 -1  1  1  1  1  1  1 -1  1
  1 -1 -1  1  1  1 -1  1  1 -1  1  1 -1 -1 -1  1  1 -1  1 -1 -1  1 -1  1
 -1  1  1 -1 -1  1 -1 -1 -1  1 -1  1  1 -1 -1 -1  1  1 -1 -1 -1 -1  1  1
 -1  1 -1  1  1  1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1  1 -1  1 -1
  1  1  1  1  1  1  1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1  1  1 -1 -1  1 -1
 -1 -1  1 -1 -1  1  1 -1  1 -1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1  1
 -1 -1  1  1 -1  1  1 -1 -1  1 -1  1  1 -1  1 -1 -1 -1  1 -1  1  1  1 -1
 -1  1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1 -1  1  1  1 -1 -1 -1
 -1 -1 -1  1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1  1  1 -1 -1  1 -1  1  1 -1
 -1 -1  1 -1 -1  1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1
  1  1  1 -1 -1 -1 -1 -1  1  1  1 -1  1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1
 -1  1  1 -1 -1  1 -1 -1  1 -1 -1 -1 -1  1 -1 -1  1

In [9]:
from model import SVM
model2 = SVM(kernel='linear', learning_rate=0.001, max_itr=1900, debug=True, lambda_param=0.01)
model2.fit(x_train, y_train)

predict = model2.predict(x_test)
predict_validate = model2.predict(x_validate)

print(model2.score(x_validate, y_validate))

for i in range(len(predict)):
    print(predict[i], y_test[i])
print(model2.score(x_test, y_test))

-- Parameter --
C: 1
Learning Rate: 0.001
Lambda Param: 0.01
N_Iters: 1900
---------------
Gradient 2033000 steps.
0.738562091503268
1.0 1
1.0 -1
-1.0 1
1.0 1
-1.0 -1
1.0 1
-1.0 -1
-1.0 -1
-1.0 -1
1.0 1
-1.0 -1
-1.0 -1
1.0 1
1.0 -1
-1.0 -1
-1.0 -1
1.0 1
-1.0 -1
-1.0 -1
-1.0 1
-1.0 -1
-1.0 -1
1.0 1
1.0 1
-1.0 -1
1.0 -1
-1.0 -1
1.0 1
1.0 1
1.0 1
-1.0 -1
-1.0 1
1.0 1
-1.0 1
1.0 -1
1.0 1
-1.0 -1
-1.0 1
-1.0 -1
1.0 1
1.0 -1
-1.0 -1
-1.0 -1
1.0 -1
1.0 1
-1.0 1
-1.0 -1
-1.0 1
-1.0 -1
1.0 1
1.0 1
1.0 1
-1.0 -1
-1.0 -1
-1.0 -1
-1.0 1
1.0 1
-1.0 -1
-1.0 -1
-1.0 1
1.0 1
1.0 1
1.0 -1
-1.0 -1
1.0 -1
1.0 1
1.0 -1
1.0 1
-1.0 -1
1.0 1
-1.0 1
1.0 1
-1.0 1
-1.0 -1
-1.0 -1
1.0 1
1.0 1
1.0 -1
1.0 -1
-1.0 -1
1.0 1
-1.0 -1
1.0 1
1.0 1
-1.0 -1
-1.0 -1
-1.0 -1
1.0 1
1.0 1
-1.0 -1
1.0 1
-1.0 -1
-1.0 1
1.0 1
-1.0 -1
-1.0 -1
-1.0 -1
1.0 1
1.0 1
-1.0 -1
-1.0 -1
1.0 1
-1.0 1
-1.0 -1
1.0 -1
1.0 -1
-1.0 -1
1.0 1
1.0 1
-1.0 -1
-1.0 -1
-1.0 -1
1.0 1
1.0 1
-1.0 1
1.0 -1
-1.0 -1
-1.0 -1
1.0 -1
1.0 1
-1.0 -1
1.0 1
-1.0 1